In [54]:
import gradio as gr
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, set_seed
import torch
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage

In [66]:
model_id = "gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(model_id, add_prefix_space=True)
model = AutoModelForCausalLM.from_pretrained(model_id)
set_seed(20)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=200, device_map='auto')

hf = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# hf = HuggingFacePipeline.from_model_id(
#     model_id="gpt2",
#     task="text-generation",
#     device_map='auto',
#     pipeline_kwargs={"max_new_tokens": 20}
# )

# textGen = pipeline("text-generation", model='EleutherAI/gpt-neo-1.3B', framework='pt')
# Godzilla is a fictional giant monster originating from Japanese cinema. It first appeared
# in the 1954 film "Godzilla," directed by Ishirō Honda. Godzilla is often depicted as a
# prehistoric sea monster awakened and empowered by nuclear radiation, symbolizing the
# consequences of atomic weapons. The character has since become a cultural icon,
# appearing in numerous films, television series, and other media.

# prefix = """
# Here are examples between a human and AI. The human provides a question about Godzilla, and
# the AI provides a single sentence with easy to read words The sentence does not have to include the 
# original question. For example:
# """

# examples = [
#     {
#         "query": "Who is godzilla?",
#         "answer": "Godzilla is a fictional giant monster originating from Japanese cinema. It first appeared in the 1954 film 'Godzilla,' directed by Ishirō Honda. Godzilla is often depicted as a prehistoric sea monster awakened and empowered by nuclear radiation, symbolizing the consequences of atomic weapons. The character has since become a cultural icon, appearing in numerous films, television series, and other media."
#     },
#     {
#         "query": "What is Godzilla's Power?",
#         "answer": "Godzilla's powers vary depending on the iteration, but he is typically depicted as having immense size and strength, able to withstand powerful attacks and cause widespread destruction. In some versions, Godzilla can also emit atomic breath, a powerful beam of radiation that he uses to attack his enemies."
#     },
#     {
#         "query": "Who is Godzilla's friend?",
#         "answer": "In various movies and series, Godzilla has been portrayed both as a destructive force and a protector of humanity. Throughout the franchise, Godzilla has formed alliances with other monsters like Mothra, Rodan, and King Kong to combat common foes. Mothra, in particular, is often depicted as a friend and ally of Godzilla, aiding him in battles against powerful adversaries."
#     }
# ]

# format = """
# Human: {query}
# AI: {answer}
# """
# example_template = PromptTemplate(
#     input_variables=["query", "answer"],
#     template=format
# )

# suffix = """
# Human: {query}
# AI: 
# """

# prompt_template = FewShotPromptTemplate(
#     examples=examples,
#     example_prompt=example_template,
#     input_variables=["query"],
#     prefix=prefix,
#     suffix=suffix,
#     example_separator="\n\n"
# )

In [67]:
# Define the format for the template.
format = """
Godzilla is a fictional giant monster originating from Japanese cinema. It first appeared
in the 1954 film "Godzilla," directed by Ishirō Honda. Godzilla is often depicted as a
prehistoric sea monster awakened and empowered by nuclear radiation, symbolizing the
consequences of atomic weapons. The character has since become a cultural icon,
appearing in numerous films, television series, and other media.
"""

# Define the query
query = "Who is Godzilla?"

# Combine the format and query into a prompt
prompt = f"{format}\n\n{query}\n"

# Generate text using the prompt
generated_text = pipe(prompt, max_new_tokens=30)[0]["generated_text"]

# Remove the format and extract only the answer
answer = generated_text.replace(prompt, "").strip()

# Print the answer
print(answer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=30) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


The name Godzilla comes from a Japanese expression meaning "the huge guy." He is a towering giant in both size and mass,

and also


In [72]:
# Create a Gradio Blocks instance for structuring the interface
app = gr.Blocks(fill_height=True)

#ChatBot function
def respond(message, chat_history):
    prompt = f"{format}\n\n{message}\n"
    generated_text = pipe(prompt, max_new_tokens=30)[0]["generated_text"]
    bot_message = generated_text.replace(prompt, "").strip()
    chat_history.append((message, bot_message))
    return "", chat_history

#Forecast bot function
def pred(info, query, tokens):
    # Combine the format and query into a prompt
    prompt = f"{info}\n\n{query}\n"

    # Generate text using the prompt
    generated_text = pipe(prompt, max_new_tokens=tokens)[0]["generated_text"]

    # Remove the format and extract only the answer
    answer = generated_text.replace(prompt, "").strip()
    return answer

# Add tabs to the Gradio Blocks
with app:
    #A chatbot that will be used to talk with our first model we trained
    with gr.Tab("ChatBot"):
        chatbot = gr.Chatbot()
        msg = gr.Textbox(placeholder="Ask Your Question", show_label=False, scale=1)
        clear = gr.ClearButton([msg, chatbot])
        msg.submit(respond, [msg, chatbot], [msg, chatbot])


    #A basic input/output which will take some text and output a prediction that will be used with our second Model
    with gr.Tab("Forecast Bot"):
        with gr.Row():

        
            gr.Interface(fn=pred, allow_flagging='never',
                            inputs=[
                            gr.Textbox(lines=10, placeholder="Input", label="Add your info to the AI"),
                            gr.Textbox(lines=1, placeholder="Input", label="Question"),
                            gr.Slider(2, 100, value=4, label="How Creative do you want your answer.", info="Choose between 2 and 100"),
                            #gr.CheckboxGroup(["USA", "Japan", "Canada"], label="Countries", info="Where is your prediction"),
                            ],
                            outputs=gr.Textbox(lines=12, placeholder="Output", label="Prediction", show_copy_button=True, interactive=False))
                

# Launch the app
app.launch()

Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=60.1) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
